In [1]:
import pandas as pd
import numpy as np
import cv2
import mediapipe as mp
import tensorflow as tf

In [14]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
draw = mp.solutions.drawing_utils

In [3]:
def capture_datapoints():
    vid = cv2.VideoCapture(0)
    vid.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    label = input("\nEnter the Gesture Name: ")

    datapoints = []
    data_count = 1

    print(f'\nInitiating Capture for {label}')

    while True:
        _, frame = vid.read()

        frame = cv2.flip(frame, 1)
        y,x,_ = frame.shape


        rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = hands.process(rgb_img)

        landmarks = {'label': label}
        if res.multi_hand_landmarks:
            #? Drawing landmarks on the frame
            for handslms in res.multi_hand_landmarks:
                draw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)
                
                lndmarks = handslms.landmark
                for i in range(len(lndmarks)):
                    landmarks[f'x{i}'] = lndmarks[i].x * x
                    landmarks[f'y{i}'] = lndmarks[i].y * y

        cv2.imshow("frame", frame)

        key = cv2.waitKey(0)
        if key == ord("q"):
            break

        elif key == ord("s"):
            datapoints.append(landmarks)
            print(f'{data_count}) {landmarks}')
            data_count += 1

        elif key == ord("n"):
            label = input("\nEnter the Gesture Name: ")
            print(f"\nInitiating Capture for {label}")
            data_count = 1

    vid.release()
    cv2.destroyAllWindows()

    return datapoints

In [4]:
def refine_data(data):
    res = []
    count = 0
    for i in data:
        if len(i) == 43:
            res.append(i)
        else:
            count+=1

    print(f'{count} data points removed')
    return res

In [5]:
data = capture_datapoints()


Initiating Capture for backward
1) {'label': 'backward', 'x0': 359.42771911621094, 'y0': 328.9152145385742, 'x1': 303.5209846496582, 'y1': 312.05984115600586, 'x2': 254.62377548217773, 'y2': 273.3410453796387, 'x3': 235.57308197021484, 'y3': 226.00553512573242, 'x4': 253.04927825927734, 'y4': 185.18215656280518, 'x5': 262.1395492553711, 'y5': 207.45497703552246, 'x6': 232.95135498046875, 'y6': 179.4926404953003, 'x7': 244.13837432861328, 'y7': 225.41855335235596, 'x8': 259.12899017333984, 'y8': 235.1491355895996, 'x9': 299.2602348327637, 'y9': 200.12100219726562, 'x10': 271.9361877441406, 'y10': 171.2173891067505, 'x11': 283.55224609375, 'y11': 233.35071086883545, 'x12': 297.8164863586426, 'y12': 233.57425689697266, 'x13': 337.24842071533203, 'y13': 199.04009342193604, 'x14': 312.2331428527832, 'y14': 181.66491508483887, 'x15': 320.52215576171875, 'y15': 240.48903465270996, 'x16': 335.0226593017578, 'y16': 241.0348892211914, 'x17': 377.2575378417969, 'y17': 201.5804100036621, 'x18': 3

In [6]:
data = refine_data(data)

0 data points removed


In [7]:
df = pd.DataFrame(data)
df.head()

,label,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x16,y16,x17,y17,x18,y18,x19,y19,x20,y20
0,backward,359.427719,328.915215,303.520985,312.059841,254.623775,273.341045,235.573082,226.005535,253.049278,...,335.022659,241.034889,377.257538,201.580410,347.666016,188.128138,348.775787,229.443612,361.615906,235.090041
1,backward,509.933243,285.574322,459.407730,270.350904,411.843796,222.397356,405.741272,168.445773,444.696236,...,492.881889,191.992364,540.499954,158.384714,530.082741,142.165232,526.425514,185.967178,527.350426,192.073917
2,backward,499.627571,294.389219,447.831421,280.322542,396.488380,232.913361,386.074600,178.362579,423.873482,...,481.609116,195.564208,527.345619,163.681698,514.186287,146.727133,511.371918,190.645266,513.984909,196.185665
3,backward,328.262901,373.813362,264.845066,325.947561,240.076389,280.745201,253.910408,243.339386,288.941212,...,343.928528,269.921494,392.434692,241.881752,374.672775,239.222975,371.190109,258.067961,371.198006,276.084194
4,backward,358.238640,342.038097,405.587807,302.241898,431.546974,264.183168,420.106163,228.448391,393.769455,...,329.042511,255.463943,282.651215,230.157409,292.885246,235.344486,298.877983,254.480667,302.675457,269.565926


In [8]:
df.shape

(60, 43)

In [ ]:
print(blabla)

In [29]:
#df.to_csv('hand_gestures.csv', index=False)

In [9]:
old_df = pd.read_csv('hand_gestures.csv')

In [10]:
new_df = pd.concat([old_df, df], ignore_index=True)

In [11]:
new_df.shape

(370, 43)

In [13]:
#new_df.to_csv('hand_gestures.csv', index=False)

In [26]:
augmented_df = new_df.copy()

for i in range(-50, 51, 5):
    augmented_data = new_df.select_dtypes(exclude='object') + i
    augmented_data['label'] = new_df['label']
    augmented_df = pd.concat([augmented_df, augmented_data], ignore_index=True)

In [27]:
augmented_df.shape

(8140, 43)

In [29]:
augmented_df.to_csv('hand_gestures.csv', index=False)